G - generative
D - Discriminator

G neural network, takes random noise signal and output some image, theif trying to steal
D another neural network, rivaling generator, police trying to detect, understands what the generator is trying to do

### APPLICATIONS

generating images<br>
image modification<br>
super resolution<br>
assisting artists<br>
photo-realistic images<br>
speech recognition<br>
face aging

image generation is the main application

## import libraries

In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.autograd import Variable

In [ ]:
batchsize = 64
imagesize = 64

In [ ]:
transform = transforms.Compose([transforms.Scale(imagesize),
                                transforms.ToTensor(),
                                transforms.Normalize(
                                    (0.5, 0.5, 0.5),
                                    (0.5, 0.5, 0.5)
                                ),
                               ]) # We create a list of transformations (scaling, tensor conversion, normalization) to apply to the input images.

### load dataset

In [ ]:
! ls ../datasets/gan_cifar10/data

In [ ]:
dataset = dset.CIFAR10(root = '../datasets/gan_cifar10/data',
                      download = True, transform = transform)

dataloader gives us data batch to batch

In [ ]:
dataloader = torch.utils.data.DataLoader(dataset, batch_size = batchsize,
                                        shuffle=True, num_workers=2)

weights will take neural network and initialize weights for both networks

In [ ]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)

defining generator, its neural network and forward function

In [ ]:
class G(nn.Module):
    
    def __init__(self):
        ''' initialize the generator neural network'''
        super(G, self).__init__()
        
        self.main = nn.Sequential( # We create a meta module of a neural network that will contain a sequence of modules (convolutions, full connections, etc.).
            nn.ConvTranspose2d(100, 512, 4, 1, 0, bias = False), # We start with an inversed convolution.
            nn.BatchNorm2d(512), # We normalize all the features along the dimension of the batch.
            nn.ReLU(True), # We apply a ReLU rectification to break the linearity.
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(256), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(128), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.BatchNorm2d(64), # We normalize again.
            nn.ReLU(True), # We apply another ReLU.
            nn.ConvTranspose2d(64, 3, 4, 2, 1, bias = False), # We add another inversed convolution.
            nn.Tanh() # We apply a Tanh rectification to break the linearity and stay between -1 and +1.
        )
        
    def forward(self, input):
        ''' it takes in the network, forward propagate and returns 
        the generated image'''
        output = self.main(input) # forward propagate through whole nw
        return output # return o/p containint generated image

##### creating Generator object 

In [ ]:
netG = G() # create generator object
netG.apply(weights_init) # initialize all the weights of its neural network

## defining discriminator, its neural network and forward function

In [ ]:
class D(nn.Module):
    
    def __init__(self):
        '''initialize discriminator neural network'''
        super(D, self).__init__()
        
        self.main = nn.Sequential(
            
        nn.Conv2d(3, 64, 4, 2, 1, bias = False),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(64, 128, 4, 2, 1, bias = False),
        nn.BatchNorm2d(128),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(128, 256, 4, 2, 1, bias = True),
        nn.BatchNorm2d(256),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(256, 512, 4, 2, 1, bias = False),
        nn.BatchNorm2d(512),
        nn.LeakyReLU(0.2, inplace = True),
        nn.Conv2d(512, 1, 4, 1, 0, bias = False),
        nn.Sigmoid()
            
        )
        
    def forward(self, input):
        output = self.main(input) # forward propagate through whole nw
        return output.view(-1) # return output value b/w 0 or 1

##### creating discriminator object

torch.module.apply(fn) Applies fn recursively to every submodule (as returned by .children()) as well as self. Typical use includes initializing the parameters of a model

In [ ]:
netD = D() # create discriminator object
netD.apply(weights_init) # initialize all the weightsof its neural network

### training DCGANs 

In [ ]:
criterion = nn.BCELoss() # Criterion will measure the error between prediction and target

In [ ]:
# optimizer for the discriminator
optimizerD = optim.Adam(netD.parameters(), lr= 0.0002,
                        betas = (0.5, 0.999))

In [ ]:
# optimizer for the generator
optimizerG = optim.Adam(netG.parameters(), lr = 0.0002,
                       betas = (0.5, 0.999))

In [ ]:
total_epochs = 25

In [ ]:
for epoch in range(total_epochs):
    for i, data in enumerate(dataloader, 0): # iterate over image of dataset
        # 1st step: Updating weights of neural network of discriminator
        netD.zero_grad() # we initialize to 0 the gradients of the discriminator w.r.t its weights
        # Training discriminator with real image of the dataset which will be used to train the discriminator
        real, _ = data # we get a real image of dataset which will train the discriminator
        input = Variable(real) # we wrap it in a Variable
        target = Variable(torch.ones(input.size()[0])) # we get the target
        output = netD(input) # we forward propagate this real image into nn of discriminator to get prediction between 1 or 0
        errD_real = criterion(output, target) # we compute loss b/w predictions op and target equal to 1
        
        # Training the discriminator with a fake image generated by generator
        noise = Variable(torch.randn(input.size()[0], 100, 1, 1)) # made a random input vector (noise) of the generator
        fake = netG(noise) # we forward propagate this random input vector into nn of generator to get some fake generated image
        target = Variable(torch.zeros(input.size()[0])) # we get the target
        output = netD(fake.detach()) # we forward propagate the fake generated image into the network of the discriminator to get the prediction between 0 or 1
        errD_fake = criterion(output, target) # we compute thhe loss b/w prediction (output) and the target (equal to 0)
        
        #BCK PROPAGATING THE TOTAL ERROR
        errD = errD_real + errD_fake # we compute total error of discriminator
        errD.backward() # we backpropagate the loss error by computing the gradients of total respect to the weights of the discriminator
        optimizerD.step() #we apply the optimizer to update the weights according to how much they are responsible for the loss of the discriminator
        
        # 2nd step: updating the weights of the neural network of the generator
        netG.zero_grad() # initialize to the 0 the gradients of the generator with respect to the weights
        target = Variable(torch.ones(input.size()[0])) # we get the target
        output = netD(fake) # we forward propagate the fake generated images into the neural of the discriminator to get the prediction (val bw 0 and 1)
        errG = criterion(output, target) # we compute the loss error by computing the gradients of the total with respoect to the weights of the generator
        optimizerG.step() # we apply optimizer to update the weights according to how much they are responsible for the loss error of the generator
        
        # 3rd Step: printing the losses and saving the real image and the generated image of the minibatch every 100 steps
        print('[%d/%d][%d/%d] Loss_D: %.4f Loss_G: %.4f' % (epoch, 25, i, len(dataloader), errD.item(), errG.item())) # We print les losses of the discriminator (Loss_D) and the generator (Loss_G).
        
        if i % 100 == 0:
            vutils.save_image(real, '%s/real_samples.png' % "../datasets/gan_cifar10/results/", normalize = True) # We save the real images of the minibatch.
            fake = netG(noise) #We get our fake generated images.
            vutils.save_image(fake.data, '%s/fake_samples_epoch_%03d.png' % ("../datasets/gan_cifar10/results/", epoch), normalize = True) # We also save the fake generated images of the minibatch.
        